In [1]:
import scipy
import scipy.io as sio
import scipy.fftpack
import scipy.signal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.transforms import _read_fs_xfm
import plotly.graph_objs as go
import nibabel as nb

In [26]:
# Load electrodes position
elec_df = pd.read_csv('/home/mcesped/scratch/code/sEEGPrep/Results/run_all/bids/sub-090/ses-005/ieeg/sub-090_ses-005_task-full_run-01_regions_native_space.tsv', sep='\t')

In [27]:
elec_df.head()

,type,label,x,y,z,group,Label ID,Label,R,G,B
0,SEEGA,LAHc1-2,-22.2600,23.3750,47.0115,LAHc,17,Left-Hippocampus,220,216,20
1,SEEGA,LAHc2-3,-27.0905,22.3880,47.5035,LAHc,17,Left-Hippocampus,220,216,20
2,SEEGA,LAHc3-4,-31.9305,21.6575,47.9330,LAHc,17,Left-Hippocampus,220,216,20
3,SEEGA,LAHc4-5,-36.9285,21.1135,48.5630,LAHc,2,Left-Cerebral-White-Matter,245,245,245
4,SEEGA,LAHc5-6,-41.9405,20.3045,48.8780,LAHc,2,Left-Cerebral-White-Matter,245,245,245


In [28]:
coordinates_contrast = elec_df[['x','y','z']].to_numpy()
coordinates_contrast[0:10,:]

array([[-22.26  ,  23.375 ,  47.0115],
       [-27.0905,  22.388 ,  47.5035],
       [-31.9305,  21.6575,  47.933 ],
       [-36.9285,  21.1135,  48.563 ],
       [-41.9405,  20.3045,  48.878 ],
       [-46.9095,  19.6385,  49.279 ],
       [-51.8355,  19.295 ,  49.9665],
       [-56.704 ,  18.9515,  50.6255],
       [-61.5155,  18.5505,  51.3415],
       [-19.0425,  31.3875,  44.5425]])

In [11]:
def readRegMatrix(trsfPath):
	with open(trsfPath) as (f):
		return np.loadtxt(f.readlines())

In [30]:
# Transform to non-contrast space
t1_transform=readRegMatrix('/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/atlasreg/sub-P090/sub-P090_acq-noncontrast_desc-rigid_from-noncontrast_to-contrast_type-ras_xfm.txt')

In [31]:
# Transform from contrast mri ras to non-contrast MRI ras
coordinates_non_contr = mne.transforms.apply_trans(t1_transform, coordinates_contrast)
coordinates_non_contr[0:10,:]

array([[-23.76558978,  44.68103676, -11.85390924],
       [-28.50219986,  43.42016759, -11.42543681],
       [-33.26468098,  42.41753181, -11.04493166],
       [-38.19504328,  41.58154553, -10.45380609],
       [-43.12404073,  40.49795488, -10.19239605],
       [-48.0189211 ,  39.55434419,  -9.8369173 ],
       [-52.89009135,  38.91862865,  -9.17690519],
       [-57.70424055,  38.28744373,  -8.54528068],
       [-62.45786358,  37.59891641,  -7.85986867],
       [-21.08328839,  52.95864647, -13.86315137]])

In [32]:
# # Transform to MNI
# subjects_dir = '/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer'
# mri_mni_trans = mne.read_talxfm('P079', subjects_dir)

In [57]:
colors = elec_df[['R','G','B']].to_numpy()
colors[0:10,:]

array([[220, 216,  20],
       [220, 216,  20],
       [220, 216,  20],
       [245, 245, 245],
       [245, 245, 245],
       [140, 220, 220],
       [160, 100,  50],
       [160, 100,  50],
       [245, 245, 245],
       [103, 255, 255]])

In [45]:
labels = elec_df['Label'].tolist()
labels[0:10]

['Left-Hippocampus',
 'Left-Hippocampus',
 'Left-Hippocampus',
 'Left-Cerebral-White-Matter',
 'Left-Cerebral-White-Matter',
 'ctx-lh-superiortemporal',
 'ctx-lh-middletemporal',
 'ctx-lh-middletemporal',
 'Left-Cerebral-White-Matter',
 'Left-Amygdala']

In [37]:
# Transform from RAS to MNI space
# MNI_transform = _read_fs_xfm('/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer/sub-P090/mri/transforms/talairach.xfm')
MNI_transform=readRegMatrix('/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/atlasreg/sub-P090/sub-P090_desc-affine_from-subject_to-MNI152NLin2009cSym_type-ras_xfm.txt')

In [38]:
MNI_transform

array([[ 9.204804e-01,  7.110123e-02, -1.582734e-02, -3.150426e+00],
       [-6.151614e-02,  9.292214e-01, -1.319229e-01,  3.201959e+01],
       [ 1.048091e-02,  1.247004e-01,  8.477862e-01,  6.228736e+01],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

In [23]:
# using FS transform
coordinates_MNI = mne.transforms.apply_trans(MNI_transform[0], coordinates_non_contr)
coordinates_MNI[0:10,:]

array([[-20.10290567,  -6.89039594, -26.95168436],
       [-24.63279631,  -8.78470821, -25.42142947],
       [-28.96339742, -10.51614727, -24.05333921],
       [-33.24619462, -12.26733769, -22.50565149],
       [-37.55514396, -14.14406506, -21.09610578],
       [-41.83645534, -16.25398905, -20.00159364],
       [-46.23776576, -18.46496461, -18.94899813],
       [-50.72939659, -20.57074025, -17.85021204],
       [-55.17555381, -22.57345789, -16.67564897],
       [-34.37667308,  16.86749459, -20.64702814]])

In [39]:
# it has to be the inverted version bc the txt files have actually the inv transform (based on its name, like the non_contrast to contrast case)
coordinates_MNI = mne.transforms.apply_trans(np.linalg.inv(MNI_transform), coordinates_contrast) 
coordinates_MNI[0:10,:]

array([[-20.03824423, -12.88352095, -15.87577239],
       [-25.17255969, -14.16738867, -15.04312011],
       [-30.33904662, -15.19314846, -14.32175647],
       [-35.69045298, -16.00109256, -13.39364681],
       [-41.03707608, -17.13975744, -12.78850669],
       [-46.34857521, -18.11135356, -12.10693418],
       [-51.63853093, -18.69462796, -11.14480705],
       [-56.86654944, -19.27866739, -10.21695007],
       [-62.02774894, -19.90975258,  -9.21576529],
       [-17.24898864,  -4.66601644, -20.03125409]])

In [49]:
from nilearn import plotting  
view = plotting.view_markers( 
      coordinates_MNI, colors, marker_size=10, marker_labels=labels) 
view.save_as_html("surface_plot.html")

## Greydon's code

In [59]:
AXIS_CONFIG = {
    "showgrid": False,
    "showline": False,
    "ticks": "",
    "title": "",
    "showticklabels": False,
    "zeroline": False,
    "showspikes": False,
    "spikesides": False,
    "showbackground": False,
}

LAYOUT = {
	"scene": {f"{dim}axis": AXIS_CONFIG for dim in ("x", "y", "z")},
	"paper_bgcolor": "#fff",
	"hovermode": False,
	"showlegend":True,
	"legend":{
		"itemsizing": "constant",
		"groupclick":"togglegroup",
		"yanchor":"top",
		"y":0.8,
		"xanchor":"left",
		"x":0.05,
		"title_font_family":"Times New Roman",
		"font":{
			"size":20
		},
		"bordercolor":"Black",
		"borderwidth":1
	},
	"margin": {"l": 0, "r": 0, "b": 0, "t": 0, "pad": 0},
}

CAMERAS = {
    "left": {
        "eye": {"x": -1.5, "y": 0, "z": 0},
        "up": {"x": 0, "y": 0, "z": 1},
        "center": {"x": 0, "y": 0, "z": 0},
    },
    "right": {
        "eye": {"x": 1.5, "y": 0, "z": 0},
        "up": {"x": 0, "y": 0, "z": 1},
        "center": {"x": 0, "y": 0, "z": 0},
    },
    "dorsal": {
        "eye": {"x": 0, "y": 0, "z": 1.5},
        "up": {"x": 0, "y": 1, "z": 0},
        "center": {"x": 0, "y": 0, "z": 0},
    },
    "ventral": {
        "eye": {"x": 0, "y": 0, "z": -1.5},
        "up": {"x": 0, "y": 1, "z": 0},
        "center": {"x": 0, "y": 0, "z": 0},
    },
    "anterior": {
        "eye": {"x": 0, "y": 1.5, "z": 0},
        "up": {"x": 0, "y": 0, "z": 1},
        "center": {"x": 0, "y": 0, "z": 0},
    },
    "posterior": {
        "eye": {"x": 0, "y": -1.5, "z": 0},
        "up": {"x": 0, "y": 0, "z": 1},
        "center": {"x": 0, "y": 0, "z": 0},
    },
}


In [53]:
t1_obj = nb.load('/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer/sub-P090/mri/T1.mgz')
Torig = t1_obj.header.get_vox2ras_tkr()
#fs_transform=(t1_obj.affine-Torig)+np.eye(4)
fs_transform=np.dot(t1_obj.affine, np.linalg.inv(Torig))

lh_pial = '/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer/sub-P090/surf/lh.pial.T1'
rh_pial = '/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer/sub-P090/surf/rh.pial.T1'
verl,facel=nb.freesurfer.read_geometry(lh_pial)
verr,facer=nb.freesurfer.read_geometry(rh_pial)

all_ver = np.concatenate([verl, verr], axis=0)
all_face = np.concatenate([facel, facer+verl.shape[0]], axis=0)
surf_mesh = [all_ver, all_face]

all_ver_shift=(mne.transforms.apply_trans(fs_transform, all_ver))


all_ver_shift=(mne.transforms.apply_trans(np.linalg.inv(t1_transform), all_ver_shift))

lh_sulc = '/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer/sub-P090/surf/lh.sulc'
rh_sulc = '/home/mcesped/projects/ctb-akhanf/cfmm-bids/Khan/clinical_imaging/epi_iEEG/derivatives/fastsurfer/sub-P090/surf/rh.sulc'
lh_sulc_data = nb.freesurfer.read_morph_data(lh_sulc)
rh_sulc_data = nb.freesurfer.read_morph_data(rh_sulc)
bg_map = np.concatenate((lh_sulc_data, rh_sulc_data))

In [56]:
groups = elec_df['Label'].unique().tolist()
df = elec_df

In [64]:
mesh_3d = go.Mesh3d(x=all_ver_shift[:,0], y=all_ver_shift[:,1], z=all_ver_shift[:,2], i=all_face[:,0], j=all_face[:,1], k=all_face[:,2],opacity=.1,color='grey',alphahull=-10)
data=[mesh_3d]
for igroup in groups:
	idx = [i for i,x in enumerate(df['Label'].tolist()) if igroup in x]
	data.append(go.Scatter3d(
		x = df['x'][idx].values,
		y = df['y'][idx].values,
		z = df['z'][idx].values,
		name=igroup,
		mode = "markers",#"markers+text",
		# text=df['label'][idx].tolist(),
		# textfont=dict(
		# 	family="sans serif",
		# 	size=16,
		# 	color="black"
		# ),
		# textposition = "middle left",
		marker=dict(
			size=10,
			line=dict(
				width=1,
			),
			color=['rgb({},{},{})'.format(int(r),int(g),int(b)) for r,g,b in colors[idx]],
			opacity=1
			)))

fig = go.Figure(data=data)
fig.update_layout(scene_camera=CAMERAS['left'],
				  legend_title_text="Electrodes",
				  **LAYOUT)

value=[np.round(x,2) for x in np.arange(.05,.6-.05,.05)]
steps = []
for i in range(len(value)):
	step = dict(
		label = str(f"{value[i]:.2f}"),
		method="restyle",
		args=[{'opacity': [value[i]]+(len(data)-1)*[1],
			 'alphahull': [-10]+(len(data)-1)*[1]
		 }]
	)
	steps.append(step)

sliders = [dict(
	currentvalue={"visible": True,"prefix": "Opacity: ","font":{"size":16}},
	active=0,
	steps=steps,
	x=.35,y=.1,len=.3,
	pad={"t": 8},
)]

fig.update_layout(sliders=sliders)
fig.write_html('dummy2.html')